In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
key_path = "./creds/service_acc_auth.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])
bigquery_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [2]:
# query new data
union_weeks = """
SELECT * , 
FROM `scanning-database.scanning_data_backup.W21`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W22` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W23` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W24` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W25` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W26` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W27` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W28` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W29`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W30`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W31`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W32`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W33`
"""

In [3]:
# Overwriting old PowerBI source data
job_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10,
destination = 'scanning-database.PowerBI.PowerBI',
write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
) # set query limit to 10G
query_job = bigquery_client.query(union_weeks, job_config)  # API request
rows = query_job.result()  # Waits for query to finish
data_in_use = rows.to_dataframe()

In [4]:
data_in_use.head()

,Start_Time,Finish_Time,Activity,Seq_Code,Recipe_Name,Break_Reasons,Missing_Ingredients,Kitting_Line,Assembly_Batch,Event_Shift,Team_Leader,Pickers_Count,Time_Consumption,Week
0,2020-07-06 12:59:31+00:00,2020-07-06 13:00:50+00:00,TBD,None,H2,None,None,KL7,None,Monday Morning,Bayu Agus Alfian,0,1.32,2020-28
1,2020-06-12 13:42:00+00:00,2020-06-12 16:29:00+00:00,Break,None,C2,shift break,None,KL10,None,Friday Afternoon,JAMES RAY ALAY-AY;,0,167.27,2020-25
2,2020-06-06 08:17:00+00:00,2020-06-06 08:18:00+00:00,Production,None,J4,None,None,KL6,None,Saturday Morning,Test,0,0.32,2020-24
3,2020-07-11 17:53:00+00:00,2020-07-11 17:53:00+00:00,Preparation/Changeover,None,A4,None,None,KL2,None,Saturday Afternoon,Sanjay,0,0.48,2020-29
4,2020-05-20 18:11:00+00:00,2020-05-20 18:28:00+00:00,Break,None,F2,None,None,KL1,None,Wednesday Afternoon,Kim,0,17.42,2020-22


In [5]:
data_in_use.shape

(91831, 14)